In [1]:
"""Example of a simple nurse scheduling problem."""
from ortools.sat.python import cp_model
import pandas as pd
import numpy as np
from utility import JobPartialSolutionPrinter
from query_data import get_data

In [2]:
data_dict = get_data()
jobs_df = data_dict['jobs_df']
availability_df = data_dict['availability_df']
skills_df = data_dict['skills_df']
all_members = data_dict['all_members']
all_weeks = data_dict['all_weeks']
all_jobs = data_dict['all_jobs']
crucial_jobs = data_dict['crucial_jobs']
non_crucial_jobs = data_dict['non_crucial_jobs']

Testing Data Quality...
Checking if names appear in availibility file
Checking if names appear in skills_mapping file
Checking that Names in both files match
Checking that Jobs in both files match
Data Quality is great!


In [3]:
# Creates the model
model = cp_model.CpModel()

# Creates job assignment variables.
shifts = {}
for m in all_members:
    for w in all_weeks:
        for j in all_jobs:
            shifts[(m, w, j)] = model.NewBoolVar(f"shift_m{m}_w{w}_j{j}")

# Each job is assigned to exactly one member per week.
for w in all_weeks:
    for j in crucial_jobs:
        model.AddExactlyOne(shifts[(m, w, j)] for m in all_members)

# Each non crucial job is assigned to at most one member per week.
for w in all_weeks:
    for j in non_crucial_jobs:
        model.AddAtMostOne(shifts[(m, w, j)] for m in all_members)

# Each member does at most one job per week.
for m in all_members:
    for w in all_weeks:
        model.AddAtMostOne(shifts[(m, w, j)] for j in all_jobs)

# Add availability constraints
for m in all_members:
    for w in all_weeks:
        if not availability_df.loc[m, str(w)]: 
            for j in all_jobs:
                model.Add(shifts[(m, w, j)] == 0)

# Add skill constraints
for m in all_members:
    for j in all_jobs:
        if not skills_df.loc[m, j]: 
            for w in all_weeks:
                model.Add(shifts[(m, w, j)] == 0)
                
# Add constraint that members cannot be rostered thrice
for m in all_members:
    for w in all_weeks[:-2]:  # Ensure we don't go out of bounds
        for j in all_jobs:
            model.Add(
                shifts[(m, w, j)] + shifts[(m, str(int(w) + 1), j)] + shifts[(m, str(int(w) + 2), j)] <= 2
            )


# Fairness constraint: Minimize variance in assignments
total_assignments = {}
for m in all_members:
    total_assignments[m] = model.NewIntVar(0, len(all_weeks) * len(all_jobs), f"total_assignments_{m}")
    model.Add(total_assignments[m] == sum(shifts[(m, w, j)] for w in all_weeks for j in all_jobs))

# Compute the mean assignments (rounded)
avg_assignments = len(all_weeks) * len(all_jobs) // len(all_members)

# Deviation from mean
deviation = {}
for m in all_members:
    deviation[m] = model.NewIntVar(0, len(all_weeks) * len(all_jobs), f"deviation_{m}")
    model.Add(deviation[m] >= total_assignments[m] - avg_assignments)
    model.Add(deviation[m] >= avg_assignments - total_assignments[m])

squared_deviation = {}
for m in all_members:
    squared_deviation[m] = model.NewIntVar(0, (len(all_weeks) * len(all_jobs)) ** 2, f"squared_deviation_{m}")
    model.AddMultiplicationEquality(squared_deviation[m], [deviation[m], deviation[m]])

# Minimize sum of squared deviations to reduce variance
model.Minimize(-sum(total_assignments[m] for m in all_members) * 10 + sum(squared_deviation[m] for m in all_members))
                


# Creates the solver and solve.
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
solver.parameters.enumerate_all_solutions = False

solution_printer = JobPartialSolutionPrinter(
    shifts, all_members, all_weeks, all_jobs, total_assignments, verbose = 1
)

status = solver.Solve(model, solution_printer)

# Check if the optimal solution was found
if status == cp_model.OPTIMAL:
    print("\nOptimal solution found!")
    # Print the solution table
    solution_df = pd.DataFrame()
    solution_df["Job"] = all_jobs
    for w in all_weeks:
        week_list = []
        for j in all_jobs:
            job_filled = False
            for m in all_members:
                if solver.Value(shifts[(m, w, j)]):
                    week_list.append(m)
                    job_filled = True
            if not job_filled:
                week_list.append(np.nan)
        solution_df[f"Week {w}"] = week_list
    display(solution_df)
    solution_df.to_csv('data/final_solution.csv')
    # Print total assignments per member
    print("\nTotal Assignments per Member:")
    for m in all_members:
        total = solver.Value(total_assignments[m])  # Retrieve total assignments
        print(f"  - {m}: {total} jobs assigned")


elif status == cp_model.FEASIBLE:
    print("\nFeasible solution found, but not necessarily optimal.")
else:
    print("\nNo solution found.")




Solution 1


,Job,Week 0,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,Week 10,Week 11,Week 12
0,Coordinator(M),WeiXuan,YeapYuXuan,ShermanLee,FamHanFeng,YeapYuXuan,AndrewCheam,AndrewCheam,ShermanLee,ShermanLee,FamHanFeng,YeapYuXuan,ShermanLee,WeiXuan
1,Livestream#1(M),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YapMengHong,YapMengHong
2,Sound#1(M),ShermanLee,ShermanLee,SingYe,WonSen,ShermanLee,SingYe,ShermanLee,SingYe,SingYe,WonSen,SingYe,SingYe,ShermanLee
3,Sound#2(M),WonSen,SingYe,WonSen,ShermanLee,SingYe,WonSen,SingYe,WonSen,WonSen,ShermanLee,WonSen,WonSen,SingYe
4,Projectionist(M),IanMing,IanMing,HongTao,CheeFookSeng,CheeFookSeng,IanMing,CheeFookSeng,IanMing,FredKwan,FredKwan,IanMing,FredKwan,HongTao
5,Coordinator(B),Jurina,WeiXuan,TimothyKwan,AndrewCheam,Jurina,TimothyKwan,TimothyKwan,TitusLeong,Jurina,WeiXuan,TimothyKwan,TitusLeong,YeapYuXuan
6,Livestream#1(B),CheamHingGee,KooYuTang,HoSiawEng,HungSweeGeok,CheamHingGee,MuTian,KooYuTang,KooYuTang,FamHanFeng,ShawnHew,MuTian,HungSweeGeok,FamHanFeng
7,Livestream#2(B),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HungSweeGeok,NaN,NaN,HungSweeGeok
8,Sound#1(B),ShawnHew,Jeffrey,ShawnHew,ShawnHew,Jeffrey,Jeffrey,TitusLeong,Jeffrey,JairusChong,JeremyFam,TitusLeong,Jeffrey,JeremyFam
9,Sound#2(B),JairusChong,FamHanFeng,TitusLeong,Jeffrey,JairusChong,JairusChong,JeremyFam,FamHanFeng,ShawnHew,Jeffrey,Jeffrey,JairusChong,JairusChong



Total Assignments per Member:
  - FamHanFeng: 6 jobs assigned
  - ShawnHew: 5 jobs assigned
  - TitusLeong: 5 jobs assigned
  - Jeffrey: 8 jobs assigned
  - JeremyFam: 3 jobs assigned
  - MathiasChong: 0 jobs assigned
  - IanMing: 5 jobs assigned
  - Mavis: 0 jobs assigned
  - Jurina: 6 jobs assigned
  - Elly: 7 jobs assigned
  - MuTian: 2 jobs assigned
  - FuLiang: 0 jobs assigned
  - JairusChong: 6 jobs assigned
  - KooYuTang: 3 jobs assigned
  - WonSen: 9 jobs assigned
  - ShermanLee: 11 jobs assigned
  - FredKwan: 3 jobs assigned
  - CheeFookSeng: 3 jobs assigned
  - SingYe: 10 jobs assigned
  - TengZhiQian: 0 jobs assigned
  - HoSiawEng: 4 jobs assigned
  - TimothyKwan: 4 jobs assigned
  - AndrewCheam: 3 jobs assigned
  - YapMengHong: 2 jobs assigned
  - WeiXuan: 4 jobs assigned
  - CheamHingGee: 2 jobs assigned
  - HungSweeGeok: 4 jobs assigned
  - HongTao: 2 jobs assigned
  - YeapYuXuan: 4 jobs assigned

Solution 2


,Job,Week 0,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,Week 10,Week 11,Week 12
0,Coordinator(M),WeiXuan,YeapYuXuan,ShermanLee,FamHanFeng,YeapYuXuan,AndrewCheam,AndrewCheam,ShermanLee,ShermanLee,FamHanFeng,YeapYuXuan,ShermanLee,WeiXuan
1,Livestream#1(M),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YapMengHong,YapMengHong
2,Sound#1(M),WonSen,ShermanLee,SingYe,WonSen,ShermanLee,SingYe,ShermanLee,SingYe,SingYe,WonSen,SingYe,SingYe,ShermanLee
3,Sound#2(M),ShermanLee,SingYe,WonSen,ShermanLee,SingYe,WonSen,SingYe,WonSen,WonSen,ShermanLee,WonSen,WonSen,SingYe
4,Projectionist(M),IanMing,IanMing,HongTao,CheeFookSeng,CheeFookSeng,IanMing,CheeFookSeng,IanMing,FredKwan,FredKwan,IanMing,FredKwan,HongTao
5,Coordinator(B),Jurina,WeiXuan,TimothyKwan,AndrewCheam,Jurina,TimothyKwan,TimothyKwan,TitusLeong,Jurina,WeiXuan,TimothyKwan,TitusLeong,YeapYuXuan
6,Livestream#1(B),CheamHingGee,KooYuTang,HoSiawEng,HungSweeGeok,CheamHingGee,MuTian,KooYuTang,KooYuTang,FamHanFeng,ShawnHew,MuTian,HungSweeGeok,FamHanFeng
7,Livestream#2(B),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HungSweeGeok,NaN,NaN,HungSweeGeok
8,Sound#1(B),ShawnHew,Jeffrey,ShawnHew,ShawnHew,Jeffrey,Jeffrey,TitusLeong,Jeffrey,JairusChong,JeremyFam,TitusLeong,Jeffrey,JeremyFam
9,Sound#2(B),JairusChong,FamHanFeng,TitusLeong,Jeffrey,JairusChong,JairusChong,JeremyFam,FamHanFeng,ShawnHew,Jeffrey,Jeffrey,JairusChong,JairusChong



Total Assignments per Member:
  - FamHanFeng: 6 jobs assigned
  - ShawnHew: 5 jobs assigned
  - TitusLeong: 5 jobs assigned
  - Jeffrey: 8 jobs assigned
  - JeremyFam: 3 jobs assigned
  - MathiasChong: 0 jobs assigned
  - IanMing: 5 jobs assigned
  - Mavis: 0 jobs assigned
  - Jurina: 6 jobs assigned
  - Elly: 7 jobs assigned
  - MuTian: 2 jobs assigned
  - FuLiang: 0 jobs assigned
  - JairusChong: 6 jobs assigned
  - KooYuTang: 3 jobs assigned
  - WonSen: 9 jobs assigned
  - ShermanLee: 11 jobs assigned
  - FredKwan: 3 jobs assigned
  - CheeFookSeng: 3 jobs assigned
  - SingYe: 10 jobs assigned
  - TengZhiQian: 0 jobs assigned
  - HoSiawEng: 4 jobs assigned
  - TimothyKwan: 4 jobs assigned
  - AndrewCheam: 3 jobs assigned
  - YapMengHong: 2 jobs assigned
  - WeiXuan: 4 jobs assigned
  - CheamHingGee: 2 jobs assigned
  - HungSweeGeok: 4 jobs assigned
  - HongTao: 2 jobs assigned
  - YeapYuXuan: 4 jobs assigned

Solution 3


...


Total Assignments per Member:
  - FamHanFeng: 6 jobs assigned
  - ShawnHew: 5 jobs assigned
  - TitusLeong: 6 jobs assigned
  - Jeffrey: 6 jobs assigned
  - JeremyFam: 5 jobs assigned
  - MathiasChong: 0 jobs assigned
  - IanMing: 3 jobs assigned
  - Mavis: 0 jobs assigned
  - Jurina: 5 jobs assigned
  - Elly: 4 jobs assigned
  - MuTian: 5 jobs assigned
  - FuLiang: 0 jobs assigned
  - JairusChong: 5 jobs assigned
  - KooYuTang: 6 jobs assigned
  - WonSen: 9 jobs assigned
  - ShermanLee: 8 jobs assigned
  - FredKwan: 4 jobs assigned
  - CheeFookSeng: 3 jobs assigned
  - SingYe: 9 jobs assigned
  - TengZhiQian: 5 jobs assigned
  - HoSiawEng: 5 jobs assigned
  - TimothyKwan: 5 jobs assigned
  - AndrewCheam: 5 jobs assigned
  - YapMengHong: 9 jobs assigned
  - WeiXuan: 5 jobs assigned
  - CheamHingGee: 5 jobs assigned
  - HungSweeGeok: 4 jobs assigned
  - HongTao: 3 jobs assigned
  - YeapYuXuan: 4 jobs assigned

Solution 10


,Job,Week 0,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,Week 10,Week 11,Week 12
0,Coordinator(M),WeiXuan,WeiXuan,YeapYuXuan,AndrewCheam,AndrewCheam,WeiXuan,AndrewCheam,YeapYuXuan,FamHanFeng,WeiXuan,YeapYuXuan,FamHanFeng,YeapYuXuan
1,Livestream#1(M),YapMengHong,YapMengHong,NaN,YapMengHong,NaN,YapMengHong,YapMengHong,NaN,YapMengHong,YapMengHong,NaN,YapMengHong,YapMengHong
2,Sound#1(M),ShermanLee,WonSen,WonSen,SingYe,SingYe,WonSen,SingYe,ShermanLee,ShermanLee,WonSen,SingYe,ShermanLee,SingYe
3,Sound#2(M),WonSen,ShermanLee,SingYe,ShermanLee,WonSen,SingYe,WonSen,WonSen,SingYe,ShermanLee,WonSen,SingYe,ShermanLee
4,Projectionist(M),FredKwan,FredKwan,IanMing,IanMing,HongTao,CheeFookSeng,CheeFookSeng,IanMing,FredKwan,CheeFookSeng,IanMing,HongTao,HongTao
5,Coordinator(B),TimothyKwan,Jurina,TimothyKwan,Jurina,WeiXuan,Jurina,TitusLeong,AndrewCheam,Jurina,YeapYuXuan,Jurina,AndrewCheam,WeiXuan
6,Livestream#1(B),MuTian,CheamHingGee,TengZhiQian,HungSweeGeok,KooYuTang,MuTian,HungSweeGeok,CheamHingGee,KooYuTang,CheamHingGee,MuTian,KooYuTang,TengZhiQian
7,Livestream#2(B),TengZhiQian,KooYuTang,CheamHingGee,TengZhiQian,HungSweeGeok,KooYuTang,MuTian,MuTian,CheamHingGee,TengZhiQian,FamHanFeng,HungSweeGeok,HungSweeGeok
8,Sound#1(B),JairusChong,ShawnHew,ShawnHew,Jeffrey,JeremyFam,Jeffrey,JairusChong,TitusLeong,TitusLeong,ShawnHew,JairusChong,JeremyFam,JeremyFam
9,Sound#2(B),ShawnHew,FamHanFeng,TitusLeong,ShawnHew,TitusLeong,FamHanFeng,JeremyFam,JairusChong,JeremyFam,Jeffrey,Jeffrey,JairusChong,Jeffrey



Total Assignments per Member:
  - FamHanFeng: 5 jobs assigned
  - ShawnHew: 5 jobs assigned
  - TitusLeong: 5 jobs assigned
  - Jeffrey: 5 jobs assigned
  - JeremyFam: 5 jobs assigned
  - MathiasChong: 0 jobs assigned
  - IanMing: 4 jobs assigned
  - Mavis: 0 jobs assigned
  - Jurina: 5 jobs assigned
  - Elly: 5 jobs assigned
  - MuTian: 5 jobs assigned
  - FuLiang: 0 jobs assigned
  - JairusChong: 5 jobs assigned
  - KooYuTang: 5 jobs assigned
  - WonSen: 9 jobs assigned
  - ShermanLee: 8 jobs assigned
  - FredKwan: 3 jobs assigned
  - CheeFookSeng: 3 jobs assigned
  - SingYe: 9 jobs assigned
  - TengZhiQian: 5 jobs assigned
  - HoSiawEng: 5 jobs assigned
  - TimothyKwan: 5 jobs assigned
  - AndrewCheam: 5 jobs assigned
  - YapMengHong: 9 jobs assigned
  - WeiXuan: 6 jobs assigned
  - CheamHingGee: 5 jobs assigned
  - HungSweeGeok: 5 jobs assigned
  - HongTao: 3 jobs assigned
  - YeapYuXuan: 5 jobs assigned

Optimal solution found!


,Job,Week 0,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,Week 10,Week 11,Week 12
0,Coordinator(M),WeiXuan,WeiXuan,YeapYuXuan,AndrewCheam,AndrewCheam,WeiXuan,AndrewCheam,YeapYuXuan,FamHanFeng,WeiXuan,YeapYuXuan,FamHanFeng,YeapYuXuan
1,Livestream#1(M),YapMengHong,YapMengHong,NaN,YapMengHong,NaN,YapMengHong,YapMengHong,NaN,YapMengHong,YapMengHong,NaN,YapMengHong,YapMengHong
2,Sound#1(M),ShermanLee,WonSen,WonSen,SingYe,SingYe,WonSen,SingYe,ShermanLee,ShermanLee,WonSen,SingYe,ShermanLee,SingYe
3,Sound#2(M),WonSen,ShermanLee,SingYe,ShermanLee,WonSen,SingYe,WonSen,WonSen,SingYe,ShermanLee,WonSen,SingYe,ShermanLee
4,Projectionist(M),FredKwan,FredKwan,IanMing,IanMing,HongTao,CheeFookSeng,CheeFookSeng,IanMing,FredKwan,CheeFookSeng,IanMing,HongTao,HongTao
5,Coordinator(B),TimothyKwan,Jurina,TimothyKwan,Jurina,WeiXuan,Jurina,TitusLeong,AndrewCheam,Jurina,YeapYuXuan,Jurina,AndrewCheam,WeiXuan
6,Livestream#1(B),MuTian,CheamHingGee,TengZhiQian,HungSweeGeok,KooYuTang,MuTian,HungSweeGeok,CheamHingGee,KooYuTang,CheamHingGee,MuTian,KooYuTang,TengZhiQian
7,Livestream#2(B),TengZhiQian,KooYuTang,CheamHingGee,TengZhiQian,HungSweeGeok,KooYuTang,MuTian,MuTian,CheamHingGee,TengZhiQian,FamHanFeng,HungSweeGeok,HungSweeGeok
8,Sound#1(B),JairusChong,ShawnHew,ShawnHew,Jeffrey,JeremyFam,Jeffrey,JairusChong,TitusLeong,TitusLeong,ShawnHew,JairusChong,JeremyFam,JeremyFam
9,Sound#2(B),ShawnHew,FamHanFeng,TitusLeong,ShawnHew,TitusLeong,FamHanFeng,JeremyFam,JairusChong,JeremyFam,Jeffrey,Jeffrey,JairusChong,Jeffrey



Total Assignments per Member:
  - FamHanFeng: 5 jobs assigned
  - ShawnHew: 5 jobs assigned
  - TitusLeong: 5 jobs assigned
  - Jeffrey: 5 jobs assigned
  - JeremyFam: 5 jobs assigned
  - MathiasChong: 0 jobs assigned
  - IanMing: 4 jobs assigned
  - Mavis: 0 jobs assigned
  - Jurina: 5 jobs assigned
  - Elly: 5 jobs assigned
  - MuTian: 5 jobs assigned
  - FuLiang: 0 jobs assigned
  - JairusChong: 5 jobs assigned
  - KooYuTang: 5 jobs assigned
  - WonSen: 9 jobs assigned
  - ShermanLee: 8 jobs assigned
  - FredKwan: 3 jobs assigned
  - CheeFookSeng: 3 jobs assigned
  - SingYe: 9 jobs assigned
  - TengZhiQian: 5 jobs assigned
  - HoSiawEng: 5 jobs assigned
  - TimothyKwan: 5 jobs assigned
  - AndrewCheam: 5 jobs assigned
  - YapMengHong: 9 jobs assigned
  - WeiXuan: 6 jobs assigned
  - CheamHingGee: 5 jobs assigned
  - HungSweeGeok: 5 jobs assigned
  - HongTao: 3 jobs assigned
  - YeapYuXuan: 5 jobs assigned
